In [ ]:
import pandas as pd
from candecoder import decode_message_by_id

In [29]:

# opening and reading data from records.csv file
data = pd.read_csv('./records.csv')

In [30]:

# creates column titles
header = ['Timestamp', 'Variable', 'Value', 'Units']

# replaces square brackets and single quotes for correct formatting and type conversions

processed_data = pd.DataFrame(columns=header)

In [31]:
from datetime import timedelta, datetime


def extractDateInfo(rawtime):
    days = timedelta(days=rawtime)
    date = datetime(year=1899, month=12, day=30) + days
    return pd.to_datetime(date)

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3502594 entries, 0 to 3502593
Data columns (total 22 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Unnamed: 0   int64  
 1   time         float64
 2   type         object 
 3   id           object 
 4   dlc          int64  
 5   data0        object 
 6   data1        object 
 7   data2        object 
 8   data3        object 
 9   data4        object 
 10  data5        object 
 11  data6        object 
 12  data7        object 
 13  filename     object 
 14  rawtime      float64
 15  year         object 
 16  month        object 
 17  day          object 
 18  hour         object 
 19  minute       object 
 20  second       object 
 21  microsecond  object 
dtypes: float64(2), int64(2), object(18)
memory usage: 587.9+ MB


In [33]:
required_cols = ["rawtime"] + ["id"] + [f"data{n}" for n in range(0, 8)]
required_cols

['rawtime',
 'id',
 'data0',
 'data1',
 'data2',
 'data3',
 'data4',
 'data5',
 'data6',
 'data7']

In [34]:
from ast import literal_eval
def map_row(rawtime, id, *hex_bytes):
    try:
        data_bytes = [int(hex_byte, 16) for hex_byte in hex_bytes]
        values = decode_message_by_id(id, data_bytes)
        ret_values = pd.DataFrame(columns=header)
        for variable, value in values.items():
            if type(value) == tuple:
                ret_values.loc[len(ret_values)] = {
                    "Timestamp": extractDateInfo(rawtime),
                    "Variable": variable,
                    "Value": value[0],
                    "Units": value[1]
                }
            else:
                ret_values.loc[len(ret_values)] = {
                    "Timestamp": extractDateInfo(rawtime),
                    "Variable": variable,
                    "Value": value,
                    "Units": None
                }
        return ret_values
    except:
        return None

In [ ]:
from tqdm.notebook import tqdm
dfs = [map_row(*row) for row in tqdm(zip(*[data[col] for col in required_cols]))]

# if packet is not None for row in map_row(*[packet[field] for field in range(len(required_cols))])]

0it [00:00, ?it/s]

In [ ]:

for idx, row in tqdm(data.iterrows()):
    try:
        data_bytes = [int(row[f"data{n}"], 16) for n in range(0, 8)]
        values = decode_message_by_id(row["id"], data_bytes)
        for variable, value in values.items():
            processed_data.loc[len(processed_data)] = {
                "Timestamp": extractDateInfo(row["rawtime"]),
                "Variable": variable,
                "Value": value
            }
    except:
        return None
    if idx > 1000:
        break

In [ ]:
processed_data.info()

In [ ]:
processed_data